In [1]:
from scipy import fft, arange, signal
from scipy.special import logit, expit
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os
import csv
import pandas as pd
from ttictoc import TicToc
#----------------------
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
import xgboost
from sklearn.model_selection import train_test_split
#---------------------RF HP-f(x) & CV---------------
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
folder_root_Hypothesis_1 = "../../Music/hypothesis_1/"
folder_root_Hypothesis_2 = "../../Music/hypothesis_2/"

In [3]:
def read_file(path):
    """
    Reads the audio .wav file and returns the sample rate and data contents of the file.
    """
    sr, signal = wavfile.read(path)
    return sr, signal[:,0]

In [4]:
def read_all(files):
    """Branch audio file extrapolation. Uses read_file() method."""
    ret = []
    t = TicToc()
    t.tic();
    for fl in files:
        ret.append(read_file(fl))
    t.toc();
    print("Time it took to read data from files of length ", len(files), " = ", round(t.elapsed, 3), " seconds.")
    return np.array(ret)

In [5]:
def find_files(PATH, ext):
    """
    Finds all the files in a particular directory. Return only .csv files.
    """
    files = []
    for r, d, f in os.walk(PATH):
        for file in f:
            if ext in file:
                files.append(os.path.join(r, file).replace("\\","/"))
    return files

In [6]:
music_files = find_files(folder_root_Hypothesis_1, ".wav")

In [7]:
data = read_all(music_files)

Time it took to read data from files of length  13  =  3.025  seconds.


In [8]:
def frequency_sepectrum(sf, x):
    """
    Convertion of Audio from time domain to frequency domain using Fast Fourier Algorithm.
    Parameters:
    1. sf : sampling frequency (usually 44.1 KHz)
    2. x : time domain signals.
    Returns:
    1. Sampling rate.
    2. Frequency distribution (Nyquist maintained)
    """
    x = x - np.average(x)  # zero-centered.

    n = len(x)
    k = arange(n)
    tarr = n / float(sf)
    frqarr = k / float(tarr)  # two sides frequency range

    frqarr = frqarr[range(n // 2)]  # one side frequency range

    x = fft(x) / n  # fft computing and normalization
    x = x[range(n // 2)]

    return frqarr, abs(x)

In [ ]:
def plot(sample_rate, signal, state=False):
    """
    Method responsible for converting from time domain to frequency
    via the discrete fourier analysis function created in frequency_sepectrum().
    Option to form a plot of the resultant frequency domain graph with its frequency content distribution.
    Returns:
    1. Y: The frequencies.
    2. frq: The content distribution.
    """
    frq, Y = frequency_sepectrum(sample_rate, signal)
    frq = frq
    if(state == True):
        plt.plot(frq, Y)
        plt.title("Frequency (Hz) vs. Magnitude")
        plt.xlabel("Frequency (Hz)")
        plt.ylabel("Magnitude")
    return Y, frq

In [ ]:
magnitude_0, frequencies_0 = plot(data[-2][0],data[-2][1], True) #Specgram

In [ ]:
f, t, Sxx = signal.spectrogram(data[0][1], fs=data[0][0]) #Spectogram

In [ ]:
f1, t1, Sxx1 = signal.spectrogram(data[-4][1], fs=data[-4][0]) #Spectogram

In [ ]:
plt.pcolormesh(t, f, Sxx)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim(top=1000)
plt.xlim(right=60)
plt.show()

In [ ]:
t.shape, f.shape, Sxx.shape

In [ ]:
spectrum, freq, time, _ = plt.specgram(data[5][1],Fs=data[5][0])
plt.xlabel('Time')
plt.ylabel('Frequency')

## Part 2. Discrete Foureir Transformation of EEG Data on Same Music files to check for similarities

In [ ]:
folder_data = "../../data/Music/"

In [ ]:
brain_music_files = find_files(folder_data, ".csv")

In [ ]:
def remove_meta_data(PATH):
    """
    Return:
    1. Changes in Electric potential based on Unix timestamp from
        the 5 channels of the Emotiv headset. 2 channels from the Frontal Lobe, 
        1 channel from the parietal lobe, and 2 from temporal lobe.
    2. Pandas Dataframe of the data reflected from (1).
    """
    reader = csv.reader(open(PATH, "rt"), delimiter='\t')
    i = 0
    one_file_data = []
    for line in reader:
        if(i > 0):
            one_file_data.append(line)
        i += 1
    one_file_data = np.array(one_file_data)
    columns = one_file_data[0][0].split(",")[3:8]
    row_data = []
    for rows in one_file_data[1:]:
        dtx = rows[0].split(",")[3:8]
        cont = []
        for x in dtx:
            cont.append(float(x))
        row_data.append(cont)
    dataframe = pd.DataFrame(row_data, columns=columns)
    return np.array(row_data), dataframe

In [ ]:
def data_DF_dir(list_PATH):
    """
    Returns all the data from a given set of path files and its associated pandas dataframe object.
    """
    raw_data = []
    dataframes = []
    for file in list_PATH:
        rd, dfob = remove_meta_data(file)
        raw_data.append(rd)
        dataframes.append(dfob)
    return raw_data, dataframes

In [ ]:
brain_data, brain_df = data_DF_dir(brain_music_files)

In [ ]:
plt.plot(brain_data[1].T[0])

In [ ]:
brain_music_files

In [ ]:
EEG_spectrum, EEG_freq, EEG_time, _ = plt.specgram(brain_data[3].T[0],Fs=128)
plt.ylim(top=20)

In [ ]:
brain_music_files

In [ ]:
music_files

In [ ]:
EEGf, EEGt, EEGSxx2 = signal.spectrogram(brain_data[-1].T[3], fs=128) #Spectogram
plt.pcolormesh(EEGt, EEGf, EEGSxx2)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim(top=20)
plt.show()

In [ ]:
def find_similarity(a,b):
    """
    Finds the similarity between EEG data file #1 vs. EEG data file #2.
    """
    for i in range(len(a)):
        pass

In [ ]:
def percent_calculator(A, B):
    """
    Finds the distribution from SAMPLE based on TEST data.
    Uses jacard similarity.
    """
    intersection = 0
    union = 0
    for a,b in zip(A,B):
        if(bool_similarity_threshold_set(a,b)):
            intersection += 1
    union = abs(len(A) + len(B) - intersection)
    percent = intersection/float(union)
    return percent

In [ ]:
a = 100
b = 99
c = (a-b)**2
d = (a*b)/c
d*0.05

In [ ]:
Sxx.mean(), Sxx1.mean()

In [ ]:
brain_music_files

In [ ]:
EEGf, EEGt, EEGSxx1 = signal.spectrogram(brain_data[0].T[0], fs=128) #Spectogram

In [ ]:
EEGf, EEGt, EEGSxx2 = signal.spectrogram(brain_data[1].T[0], fs=128) #Spectogram

In [ ]:
clench_f, clench_t, clench_Sxx = signal.spectrogram(brain_data[3].T[0], fs=128) #Spectogram

In [ ]:
plt.pcolormesh(clench_t, clench_f, clench_Sxx)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
toge = list(clench_Sxx[0])
for x in range(1, clench_Sxx.shape[0]):
    toge.extend(list(clench_Sxx[x]))


In [ ]:
len(toge)

In [ ]:
brain_music_files[0]

In [ ]:
EEGSxx1.mean(), EEGSxx2.mean()

In [ ]:
RADIUS = 1
def bool_similarity_threshold_set(A, B):
    """
    bool function to detect for any intersection between two areas of two circles with centers @ A, B.
    """
    return bool(abs(A-B) < RADIUS)

In [ ]:
pct = 0
for i in range(EEGSxx1.shape[0]):
    pct += percent_calculator(EEGSxx1[i], EEGSxx2[i])
pct /= Sxx.shape[0]
pct

In [ ]:
# music_files

In [ ]:
EEGf, EEGt, Sxx1 = signal.spectrogram(data[-4][1], fs=data[0][0]) #Spectogram
EEGf, EEGt, Sxx2 = signal.spectrogram(data[-6][1], fs=data[0][0]) #Spectogram

In [ ]:
pct = 0
for i in range(Sxx1.shape[0]):
    pct += percent_calculator(Sxx1[i], Sxx2[i])
pct /= Sxx.shape[0]
pct

In [ ]:
magnitude, frequencies = plot(128, brain_data[3].T[0], True)